This notebook walks through the process of fitting cleaned TESS lightcurves with a model of contributions from the planet's atmospheric brightness modulation, stellar harmonics, and stellar pulsations.

**Prerequisite:** the data explored in this notebook must be lightcurves cleaned for phase curve analysis, i.e.
- Bad quality flag data have been removed
- Transits and secondary eclipses have been removed
- Data taken around momentum dumps have been removed
- Unexpected flux ramps have been removed
These clean lightcurves are produced in the TESSPhaseCurve_KELT-9b_DataCleaning.ipynb notebook.

# Background

There are many different methods of fitting a model to data. We will use ***Bayesian parameter estimation*** to fit our TESS lightcurves with our multi-component model. There are two concepts pertinent to this technique. First is the ***Bayesian approach*** which estimates parameter values by updating a prior belief about model parameters with new evidence.
1. The prior belief is represented by a ***prior distribution***, a probability distribution that describes the plausibility of the parameter having a range of values based on prior information. We will adopt either ***uniform*** or ***Gaussian*** priors as depicted below:
<div>
<img src="notebook_images/prior_distributions.png" width="1000"/>
</div> 
The x-axis represents a range of parameter values and the y-axis represents their corresponding probabilities. The uniform prior on the left assigns an equal probability to any value between $a$ and $b$. The Gaussian prior on the right assigns the highest probability to the value $\mu$.
2. The new evidence is called a ***likelihood distribution***. It attributes the probability of the parameter having a given value by comparing the resulting model to the data. The parameter value that yields the closest match to the data has the highest probability. This is quantified by the $\chi^2$ function:

    $\chi^2 = \sum_{i=1}^{N} \Big( \frac{O_i - E_i}{\sigma_i} \Big)^2$

    where $O_i$ is an observation (data point), $E_i$ is the expected value of that observation (model point), and $\sigma_i$ is the uncertainty on that observation (errorbar). The goal is to minimize $\chi^2$ because the closer the model matches the data, the smaller the difference will be and therfore $\chi^2$ will be smaller.
    
These two probabilities, the prior and the likelihood, are combined using Bayes's theorem to determine the ***posterior proability***, which is the overall probability assigned to a parameter value given the priors and the data. Bayes' Theorem says:

$p(\theta | S) = \frac{p(S | \theta) p(\theta)}{\int p(S | \theta) p(\theta)}$

Breaking it down into words, $p(\theta | S)$ is the probability of parameter value $\theta$ given a set of observations $S$, $p(S | \theta)$ is the likelihood probability, $p(\theta)$ the prior probability, and $\int p(S | \theta) p(\theta)$ is just the normalization needed so that the probabilities can be compared on the same scale and range from 0 to 1.

There are a lot of resources on Bayesian statistics on the internet. This video might help build your understanding: https://www.youtube.com/watch?v=HZGCoVF3YvM&ab_channel=3Blue1Brown.

The second concept relevant to Bayesian parameter estimation is ***parameter space sampling***. To find the best-fit parameters, we need some way of searching through different plausible values and applying Bayes' theorem to evaluate their posterior probabilities. We will do this using <a href="https://dynesty.readthedocs.io/en/latest/index.html">dynesty</a> to perform nested sampling. The technical details of nested sampling are not important for your research. The main point is that it performs a numerical integral of the posterior probability distribution over the prior volume (which set the bounds of the integral) from which you can recover posterior probabilities for all of the parameter values sampled. It does this by finding contours in parameter space of constant likelihood and honing in on the interior contours of higher likelihood (hence why it is called "nested" sampling); see <a href="https://www.youtube.com/watch?v=UnfjlA7EdjY&ab_channel=JuehangQin">this visualization</a>. Some more background on this is given here: https://dynesty.readthedocs.io/en/latest/overview.html

We will use <a href="https://dynesty.readthedocs.io/en/latest/index.html">dynesty</a> to search the parameter space to find the best-fit phase curve parameters and compute their associated uncertainties.

# Imports

You will need to have the following packages installed:
- numpy: https://numpy.org/install/
- matplotlib: https://matplotlib.org/stable/users/installing/index.html
- pandas: https://pandas.pydata.org/docs/getting_started/install.html
- astropy: https://docs.astropy.org/en/stable/install.html
- lightkurve: https://docs.lightkurve.org/about/install.html
- dynesty: https://dynesty.readthedocs.io/en/v1.2.3/